# Lab 5

In [1]:
# %pip install geemap

In [2]:
# Import packages
import ee
import geemap

In [3]:
geemap.ee_initialize()

In [4]:
# Check initialize
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

# Question 1

Visualize the USGS Watershed Boundary Dataset with outline color only, no fill color.


In [5]:
Map1 = geemap.Map()

# Load Feature Collection
watershed = ee.FeatureCollection("USGS/WBD/2017/HUC04")

# set style border
style = {'color': '0000ffff', 'width': 2, 'lineType': 'solid',
         'fillColor': '0000ff00'}

# Add text to the map
text = "Created by Pablo Miranda"
Map1.add_text(text, fontsize=20, position='bottomright')

# Add Layers
Map1.addLayer(watershed.style(**style), {},  'Watersheds')
Map1.setCenter(-95, 40, zoom=4.2)
Map1

Map(center=[40, -95], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chi…

![](https://i.imgur.com/DaU7ITK.jpeg)

# Question 2

Filter the USGS Watershed Boundary dataset and select the watershed that intersects the county of your choice.

In [6]:
Map2 = geemap.Map()

# Load counties and filter by name
borders = ee.FeatureCollection("TIGER/2018/Counties")
chatham = borders.filter(ee.Filter.eq('NAME', 'Chatham'))
chatham_37 = chatham.filter(ee.Filter.eq('STATEFP', '37')) # Select countie

# Load watershed collection
watershed = ee.FeatureCollection("USGS/WBD/2017/HUC04")
capefear = watershed.filterBounds(chatham_37)

# set style border
style = {'color': '0000ffff', 'width': 2, 'lineType': 'solid',
         'fillColor': 'FF000000'}

# Add text to the map
text = "Created by Pablo Miranda"
Map2.add_text(text, fontsize=20, position='bottomright')

# Add Layers
Map2.centerObject(capefear, 8)
Map2.addLayer(watershed.style(**style), {},  'Watersheds')
Map2.addLayer(capefear, {}, 'Cape Fear')

# Display the map
Map2

Map(center=[35.20113826203614, -78.76701818599166], controls=(WidgetControl(options=['position', 'transparent_…

![](https://i.imgur.com/nDdC7Pe.jpeg)

# Question 3

Clip the USGS 3DEP 10m DEM with the watershed that intersects the county of your choice. Display the DEM with a proper color palette and color bar.



In [7]:
Map3 = geemap.Map()

# Load watershed collection
watershed = ee.FeatureCollection("USGS/WBD/2017/HUC04")
capefear = watershed.filter(ee.Filter.eq('name', 'Cape Fear')) # Select watershed

# Load DEM
dataset = ee.Image("USGS/3DEP/10m")
elevation = dataset.select('elevation');

# Clip it
dem = elevation.clip(capefear)

# Config. visualization
vis_params = {
    'min': 0,
    'max': 350,
    'palette': 'terrain',
    }

# set style border
style = {'color': '0000ffff', 'width': 2, 'lineType': 'solid',
         'fillColor': 'FF000000'}

# Add text to the map
text = "Created by Pablo Miranda"
Map3.add_text(text, fontsize=20, position='bottomright')


# Add Layers
Map3.centerObject(capefear, 8)
Map3.addLayer(dem, vis_params, 'DEM')
Map3.addLayer(capefear.style(**style), {},  'Cape Fear')
Map3.add_colorbar(vis_params, label="Elevation (m)", layer_name='DEM',
                 orientation='horizontal', transparent_bg=False, font_size=12)

# Display the map
Map3


Map(center=[35.20113826203614, -78.76701818599166], controls=(WidgetControl(options=['position', 'transparent_…

![](https://i.imgur.com/jW05izh.jpeg)

# Question 4

Use the USGS National Land Cover Database and US Census States to create a split-panel map for visualizing land cover change (2001-2019) for a US state of your choice. Make sure you add the NLCD legend to the map.

In [8]:
Map4 = geemap.Map(basemap="CartoDB.DarkMatter")

# Load Florida border
borders = ee.FeatureCollection("TIGER/2018/States")
florida = borders.filter(ee.Filter.eq('NAME', 'Florida'))

# Load USGS landcovers
collection = ee.ImageCollection('USGS/NLCD_RELEASES/2019_REL/NLCD').select('landcover').filterBounds(florida)
vis_params = {'bands': ['landcover']}
years = collection.aggregate_array('system:index').getInfo()
clipped = collection.map(lambda image: image.clip(florida))

# Create split panel
Map4.ts_inspector(
    left_ts=clipped,
    right_ts=clipped,
    left_names=years,
    right_names=years,
    left_vis=vis_params,
    right_vis=vis_params,
    width='80px',
)

# Add text to the map
text = "Created by Pablo Miranda"
Map4.add_text(text, fontsize=20, position='bottomright')

# Add Layers
Map4.centerObject(florida, 7)
Map4.addLayer(clipped, {}, 'NLCD')

# Add legend
Map4.add_legend(
    title="NLCD Land Cover Classification", builtin_legend='NLCD', height='465px'
)

Map4

Map(center=[28.460067698576307, -82.42296617464744], controls=(WidgetControl(options=['position', 'transparent…

![](https://i.imgur.com/4RAAnPW.jpeg)


# Question 5
Download OpenStreetMap data for a city of your choice and visualize the city boundary and restaurants in the city.



In [9]:
%pip install osmnx


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 1.9 MB/s eta 0:00:00


In [10]:
import geemap.osm as osm

Map5 = geemap.Map()

# Load bounds
Map5 = geemap.Map(add_google_map=False)
gdf = osm.osm_gdf_from_geocode("Valladolid")

# Load restaurants
gdf_restaurant = osm.osm_gdf_from_address(
    address="Valladolid", tags={"amenity": "restaurant"}, dist=4000
)

# Add text to the map
text = "Created by Pablo Miranda"

# Add layers
Map5.add_text(text, fontsize=20, position='bottomright')
Map5.add_gdf(gdf, layer_name= "Valladolid")
Map5.add_gdf(gdf_restaurant, layer_name= "Restaurant")

Map5


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [11]:
gdf_restaurant

addr:city addr:housenumber addr:postcode  \
element_type osmid                                                   
node         601645609   Valladolid               13         47001   
             748583202          NaN              NaN           NaN   
             752784684          NaN              NaN           NaN   
             752784698   Valladolid               22         47015   
             753266015          NaN              NaN           NaN   
...                             ...              ...           ...   
way          871925730          NaN              NaN           NaN   
             898450909          NaN              NaN           NaN   
             906320349          NaN              NaN           NaN   
             1029892187         NaN              NaN           NaN   
relation     7529907            NaN              NaN           NaN   

                                        addr:street     amenity  \
element_type osmid                                                
node         601645609           Calle de la Pasión  restaurant   
             748583202       Plaza de Juan Pablo II  restaurant   
             752784684                          NaN  restaurant   
             752784698   Calle Monasterio de la Vid  restaurant   
             753266015                          NaN  restaurant   
...                                             ...         ...   
way          871925730                          NaN  restaurant   
             898450909                          NaN  restaurant   
             906320349                          NaN  restaurant   
             1029892187                         NaN  restaurant   
relation     7529907                            NaN  restaurant   

                        changing_table changing_table:location  \
element_type osmid                                               
node         601645609             yes           female_toilet   
             748583202             NaN                     NaN   
             752784684             NaN                     NaN   
             752784698             NaN                     NaN   
             753266015             NaN                     NaN   
...                                ...                     ...   
way          871925730             NaN                     NaN   
             898450909             NaN                     NaN   
             906320349             NaN                     NaN   
             1029892187            NaN                     NaN   
relation     7529907               NaN                     NaN   

                                         name          phone  \
element_type osmid                                             
node         601645609            Los Zagales  +34 983380892   
             748583202      La Teta y La Luna  +34 983114444   
             752784684            Pipistrello            NaN   
             752784698               Fantoche  +34 661211291   
             753266015       La Viña de Patxi            NaN   
...                                       ...            ...   
way          871925730   La Terraza Encantada            NaN   
             898450909                    NaN            NaN   
             906320349           La Fundición            NaN   
             1029892187  La Posada del Fraile            NaN   
relation     7529907            Tercer Tiempo            NaN   

                                                  website  ... addr:unit note  \
element_type osmid                                         ...                  
node         601645609   https://www.loszagales.com/tapas  ...       NaN  NaN   
             748583202                                NaN  ...       NaN  NaN   
             752784684                                NaN  ...       NaN  NaN   
             752784698               https://fantoche.es/  ...       NaN  NaN   
             753266015                                NaN  ...   

![](https://i.imgur.com/i6UNohG.jpeg)

